In [3]:
%matplotlib ipympl

import itertools
import inventory
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pandas as pd
import pykonal
import scipy.optimize

In [4]:
EVENTS = pd.read_hdf("../data/catalog.hdf5", key="events")
ARRIVALS = pd.read_hdf("../data/catalog.hdf5", key="arrivals")
ARRIVALS["location"] = ""
ARRIVALS["handle"] = ARRIVALS["network"] + "." + ARRIVALS["station"] + "." + ARRIVALS["location"] + "." + ARRIVALS["phase"]

In [83]:
class EQLocator(object):

    def __init__(
        self, 
        tt_inventory,
        delta=np.array([
            20,
            np.radians(0.2),
            np.radians(0.2),
            10
        ])
    ):
        """
        A class for locating earthquakes.
        
        Positional Arguments
        ====================
        `tt_inventory`: str
            Path to a TTInventory file created using the `compute_tts`
            program.
            
        Keyword Arguments
        =================
        `delta`: length 4 array-like
            Search range around an initial location estimate in 
            spherical coordinates (rho, theta, phi, time) with units
            of (km, rad, rad, s).
        """
        self._tti = inventory.TTInventory(tt_inventory, mode="r")
        self._delta = np.array(delta)


    def __exit__(self):
        self.__del__()


    def __del__(self):
        self.tti.f5.close()


    @property
    def arrivals(self):
        """
        Phase arrival observations used to locate event.
        
        This attribute should be set using a DataFrame with `handle`
        and `time` fields. The `handle` field should be a string 
        (e.g., SEED id code) that matches the name of the corresponding
        Dataset object in the TTInventory file. The `time` field should
        be a float representing the observed phase arrival time in as a
        UNIX/epoch timestamp (seconds since 1970-01-01T00:00:00).
        """
        return (self._arrivals)
    
    @arrivals.setter
    def arrivals(self, value):
        self._arrivals = value.set_index("handle")
        self._recompute_bounds = True
        self._tt = None
        
    @property
    def bs_arrivals(self):
        """
        Phase arrival observations with added noise for bootstrap
        uncertainty analysis.
        """
        return (self._bs_arrivals)
    
    @bs_arrivals.setter
    def bs_arrivals(self, value):
        self._bs_arrivals = value


    @property
    def bounds(self):
        """
        The bounds of the search range for the optimal location.
        """
        if not hasattr(self, "_bounds") or self._recompute_bounds is True:
            self.bounds = self.compute_bounds()
        return (self._bounds)
    
    @bounds.setter
    def bounds(self, value):
        self._recompute_bounds = False
        self._bounds = value


    @property
    def delta(self):
        """
        Search range half-widths around initial location.
        """
        return (self._delta)

    @delta.setter
    def delta(self, value):
        self._delta = value


    @property
    def method(self):
        """
        Optimization method. Defaults to scipy.optimize.differential_evolution().
        """
        if not hasattr(self, "_method"):
            self._method = scipy.optimize.differential_evolution
            
        return (self._method)
    
    @method.setter
    def method(self, value):
        self._method = value

        
    @property
    def tt(self):
        """
        A dictionary of pykonal.fields.ScalarField3D objects containing
        the traveltime data for each arrival.
        """
        if self._tt is None:
            if self._recompute_bounds is True:
                self.bounds = self.compute_bounds()
            self._tt = {handle:
                self._tti.read(
                    handle,
                    min_coords=self.bounds[0, :-1],
                    max_coords=self.bounds[1, :-1]
                )
                for handle in self._arrivals.index
           }
            
        return (self._tt)
            
    @property
    def tti(self):
        return (self._tti)

    
    def add_bs_noise(self, coords, seed=None):
        """
        Add noise to a copy of arrivals for bootstrap uncertainty
        analysis. Noise is sampled from residual distribution computed
        for provided location coordinates.
        """
        arrivals = self.arrivals.copy()
        r = self.residuals(coords, bootstrap=False)
        np.random.seed(seed)
        arrivals["time"] += np.random.choice(r, size=len(r))
        
        self.bs_arrivals = arrivals

    
    def bootstrap_analysis(self, loc, n=128, nproc=None):
        """
        Run bootstrap uncertainty analysis for location given by `loc`.
        
        Keyword Arguments
        =================
        `n`: int (default=128)
            Number of bootstrap iterations.
        `nproc`: int (default=None)
            Number of simultaneous threads to process.
        """

        seeds = np.random.randint(0, 2**32-1, n)
        args = (self.arrivals, loc, self.tti.path, self.delta, self.method)
        with mp.Pool(nproc) as pool:
            results = pool.map(
                bootstrap_target,
                ((*args, seed) for seed in seeds)
            )
            
        return (np.stack(results))


    def compute_bounds(self, loc=None):
        """
        Compute the bounds of the search range.
        
        If the optional `loc` keyword argument is provided, the lower 
        and upper bounds are simply loc - delta and loc + delta, 
        respectively. If the `loc` keyword is not provided, than an
        estimate is obtained by a brute-force grid search.
        """
        if loc is None:
            loc = self.grid_search()
        return (np.array([loc-self.delta, loc+self.delta]))


    def grid_search(self):
        """
        Return location estimate from brute-force grid search.
        """
        t0 = np.array([
            arrival["time"] - self.tti.read(index).values
            for index, arrival in self.arrivals.iterrows()
        ])
        std = np.std(t0, axis=0)
        idx_min = np.unravel_index(np.argmin(std), std.shape)
        t0 = np.mean(t0, axis=0)[idx_min]
        loc = np.array([*self.tti.nodes[idx_min], t0])

        return (loc)

    
    def locate(
        self,
        *args,
        order=2, 
        bootstrap=False,
        **kwargs
    ):
        """
        Locate the event using `target` method from scipy.optimize to
        minimize the residual.

        Keyword Arguments
        =================
        `order`: int (default=2)
            Order of the Lp-norm that will be minimized.
        `bootstrap`: bool (default=False)
            Whether to compute the location using the arrivals with
            noise added for bootstrap uncertainity analysis.
        `**kwargs`: 
            Additional keyword arguments are passed directly to
            the optimization function.
        """
        cost = lambda coords: self.norm(coords, order=order, bootstrap=bootstrap)
        return(
            self.method(
                cost,
                *args,
                bounds=self.bounds.T,
                **kwargs
            ).x
        )

    
    def norm(self, coords, order=2, bootstrap=False):
        """
        Return the Lp-norm of the residuals computed for the given
        location coordinates, `coords`.
        
        Keyword Arguments
        =================
        `order`: int (default=2)
            Order of the Lp-norm that will be minimized.
        `bootstrap`: bool (default=False)
            Whether to compute the location using the arrivals with
            noise added for bootstrap uncertainity analysis.
        """
        return (
            np.linalg.norm(
                self.residuals(coords, bootstrap=bootstrap), 
                ord=order
            )
        )


    def residuals(self, coords, bootstrap=False):
        """
        Return the residuals computed for the given location
        coordinates, `coords`.
        
        Keyword Arguments
        =================
        `bootstrap`: bool (default=False)
            Whether to compute the location using the arrivals with
            noise added for bootstrap uncertainity analysis.
        """
        if bootstrap is False:
            arrivals = self.arrivals
        elif bootstrap is True:
            arrivals = self.bs_arrivals
        else:
            raise (ValueError)
            
        tt = np.array([
            self.tt[
                handle
            ].value(
                coords[:3], 
                null=np.inf
            )
            for handle in arrivals.index
        ])
        
        return (arrivals["time"].values - coords[3] - tt)


def bootstrap_target(args):
    arrivals, loc, tti_path, delta, method, seed = args
    locator = EQLocator(tti_path, delta=delta)
    locator.method = method
    locator.arrivals = arrivals.reset_index().copy()
    locator.add_bs_noise(loc, seed=seed)

    return (locator.locate(bootstrap=True))

locator = EQLocator("../data/tts.hdf5")

In [84]:
%%time
event = EVENTS.iloc[1]
locator.arrivals = ARRIVALS.set_index("event_id").loc[event["event_id"]]
result = locator.locate()
result

CPU times: user 741 ms, sys: 102 ms, total: 843 ms
Wall time: 839 ms


array([ 6.35889040e+03,  9.84565726e-01, -2.03573818e+00,  1.19914605e+09])

In [85]:
%%time
locs = locator.bootstrap_analysis(n=16)

CPU times: user 402 ms, sys: 45.6 ms, total: 448 ms
Wall time: 5.72 s


# Development of EDT residual

In [ ]:
import itertools

def edt(self, coords):
    arrivals = self.arrivals.set_index("handle")
    pairs = list(itertools.product(arrivals.index, arrivals.index))
    r = [
        (arrivals.loc[handle1, "time"] - arrivals.loc[handle2, "time"] )
        - (self._tt[handle1].value(coords[:3], null=np.inf) - self._tt[handle2].value(coords[:3], null=np.inf))
        for handle1, handle2 in pairs
    ]
    return (r)

In [ ]:
%%time
event = EVENTS.iloc[1]
locator.arrivals = ARRIVALS.set_index("event_id").loc[event["event_id"]]

In [ ]:
locator._tt = {handle: locator.tti.read(handle) for handle in locator.arrivals.index}

In [ ]:
def residuals(self, coords, bootstrap=False):
    arrivals = locator.arrivals
    pairs = np.array(list(itertools.product(arrivals.index, arrivals.index)))
    ota = locator.arrivals.loc[pairs[:, 0], "time"].values 
    otb = locator.arrivals.loc[pairs[:, 1], "time"].values
    tts = {handle: self._tt[handle].value(coords[:3], null=np.inf) for handle in arrivals.index}
    tta = np.array([tts[handle] for handle in pairs[:, 0]])
    ttb = np.array([tts[handle] for handle in pairs[:, 1]])

    return ((ota - otb) - (tta - ttb))

In [ ]:
np.array([*nodes[i, j], 0])

In [ ]:
nodes = locator.tti.nodes[-5]
# edt_norm = np.zeros(nodes.shape[:-1])
# for i in range(edt_norm.shape[0]):
#     for j in range(edt_norm.shape[1]):
#         edt_norm[i, j] = np.linalg.norm(residuals(locator, nodes[i, j]))

l2_norm = np.zeros(nodes.shape[:-1])
for i in range(l2_norm.shape[0]):
    for j in range(l2_norm.shape[1]):
        l2_norm[i, j] = locator.norm(np.array([*nodes[i, j], loc0.x[-1]]))

In [ ]:
plt.close("all")
fig, axes = plt.subplots(ncols=2, figsize=(12, 6))
qmesh = axes[0].pcolormesh(l2_norm)
fig.colorbar(qmesh, ax=axes[0])
qmesh = axes[1].pcolormesh(edt_norm)
fig.colorbar(qmesh, ax=axes[1])

In [ ]:
%%time
loc0 = locator.differential_evolution(order=2, bootstrap=False)
loc0.x

In [ ]:
%%time
boots = np.empty((0, 4))
for i in range(100):
    locator.bootstrap_sample(loc0.x)
    loc = locator.differential_evolution(order=2, bootstrap=True)
    boots = np.vstack([boots, loc.x])

In [ ]:
np.degrees(np.std(boots[:, 1])) * 111

In [ ]:
plt.close("all")
fig, ax = plt.subplots()
ax.hist(boots[:, 0], bins=32)